# **Quality Control with the MRI Quality Check (MRIQC) BIDS App**

***

本教程将教你如何在 **Windows、macOS 和 Linux 系统** 上使用 [MRIQC](https://mriqc.readthedocs.io/en/stable/) 进行 fMRI 数据质量控制分析。

我们将使用Docker运行 MRIQC，从而保证跨平台兼容性和运行稳定性。

- **Step 1: 准备BIDS格式的数据集**  
- **Step 2: 通过Docker安装MRIQC**  
- **Step 3: 运行MRIQC的被试级分析**  
- **Step 4: 检查输出结果**  

# Step 1 准备BIDS格式的数据集

### 输入数据：符合BIDS格式的数据集


```
Bids/
├── dataset_description.json
├── participants.tsv
└── sub-001/
    ├── anat/
    │   ├── sub-001_T1w.nii.gz
    │   └── sub-001_T1w.json
    └── func/
        ├── sub-001_task-rest_bold.nii.gz
        └── sub-001_task-rest_bold.json
```


# Step 2 通过Docker安装MRIQC


In [ ]:
# 下列命令在linux、macOS、Windows上均可运行 
docker run -it nipreps/mriqc:latest --version

**注意：配置 Docker 镜像加速器**

在国内拉取 Docker 镜像时，如果出现如下错误：

```
docker: Error response from daemon: Get "https://registry-1.docker.io/v2/": read tcp 192.168.30.91:51536->44.208.254.194:443: read: connection reset by peer.
```

这表明 Docker 客户端在尝试访问 Docker Hub 或其他注册中心时出错，具体是无法获取注册中心的 API 响应。

> 🔗 参考：[知乎文章 - Docker 镜像加速](https://zhuanlan.zhihu.com/p/24228872523)


打开 Docker Desktop → Settings → Docker Engine，添加以下内容：

```json
{
  "registry-mirrors": [
    "https://docker-cf.registry.cyou",
    "https://mirror.aliyuncs.com",
    "https://mirror.baidubce.com",
    "https://docker.nju.edu.cn",
    "https://docker.mirrors.ustc.edu.cn",
    "https://mirrors.tuna.tsinghua.edu.cn"
  ]
}
```

点击 Apply & Restar。

之后在命令行/终端再次输入docker pull 的相关命令。

# Step 3 运行MRIQC的被试级分析

Windows用户可以管理员身份运行powershell，可参考以下命令：


In [ ]:
docker run -it --rm `
  # 数据输入路径：请改为自己本地运行的绝对路径，挂载为容器中的/data目录
  -v "G:\\Judge\\Bids:/data:ro" `
  # 数据输出路径：请改为自己本地运行的绝对路径，挂载为容器中的/out目录
  -v "G:\\Judge\\derivatives\\mriqc:/out" `
  # 本地已有的MRIQC镜像名称
  nipreps/mriqc `
  # 指定docker容器中数据的输入路径、输出路径，并选择分析级别为participant
  /data /out participant `
  # 指定单个被试001的数据
  --participant_label 001 `
  # 禁用自动上传至 mriqc 平台
  --no-sub

如果你使用的是 **Linux** 或 **macOS** 系统，可以直接运行 Bash 脚本来执行 MRIQC 分析：

1. 你可以将下方脚本内容保存为 `run_mriqc.sh` 文件，并在终端中赋予其执行权限后运行；
2. 或者直接使用我们已经准备好的示例脚本，路径为：`../Step_2_MRIQC/Step_2_run_mriqc.sh`。

**操作步骤如下：**

```bash
# 赋予脚本执行权限（只需执行一次）
chmod +x run_mriqc.sh

# 运行脚本，后面参数为被试编号（如 001）
./run_mriqc.sh 001

**示例脚本内容如下：**

In [ ]:
#!/bin/bash
# 定义输入数据的路径
bids_root_dir=/Volumes/ss/fMRI_Pipeline/BIDS
# 获取被试编号
subj=$1
# 设置并行线程数
nthreads=6
# 内存（GB）
mem=24 

# 输出提示信息
echo ""
echo "Running MRIQC on participant: sub-$subj"
echo ""

# 创建输出目录（如不存在）
if [ ! -d $bids_root_dir/derivatives ]; then
  mkdir $bids_root_dir/derivatives
fi  

if [ ! -d $bids_root_dir/derivatives/mriqc ]; then
  mkdir $bids_root_dir/derivatives/mriqc
fi

if [ ! -d $bids_root_dir/derivatives/mriqc/sub-${subj} ]; then
  mkdir $bids_root_dir/derivatives/mriqc/sub-${subj}
fi

# 使用 Docker 运行 MRIQC
docker run -it --rm \
  -v $bids_root_dir:/data:ro \
  -v $bids_root_dir/derivatives/mriqc/sub-${subj}:/out \
  nipreps/mriqc:latest /data /out participant \
  --participant_label $subj \
  --n_proc $nthreads \
  --mem_gb $mem \
  --float32 \
  --ants-nthreads $nthreads \
  -w /out \
  --verbose-reports \
  --no-sub

**注意：** 

- 请将 `001` 替换为你实际的被试编号。
- 如果你使用的是我们的示例脚本，只需将 `run_mriqc.sh` 中的路径替换成您的实际路径

# Step 4: 检查输出结果



MRIQC 会为每位被试的每一个扫描生成一个 **HTML 报告**，其中包含多个关于数据质量各方面的可视化图表。

在每个报告的底部，你可以点击 **[Extracted Image Quality Metrics (IQMs)](https://mriqc.readthedocs.io/en/latest/measures.html)** 查看根据扫描类型计算出的各种质量指标数值。

如果你在 `"participant"`（被试）和 `"group"`（组）两个级别都运行了 MRIQC，还会生成每种扫描类型的 **组级别报告**。

这些报告对于识别是否存在数据质量问题的被试或扫描非常有帮助。




**📎 参考资源**

- [MRIQC 官方文档](https://mriqc.readthedocs.io/en/latest/)

